In [12]:
import pandas as pd
import numpy as np
import sklearn
import re
import string
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [42]:
# import service request csv
srdf = pd.read_csv("analytics/service_requests_Train-test_set.csv")

In [55]:
# clean text

class Preprocess:
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.remove_list = stopwords.words('english')
        #self.remove_list.remove('no')
        self.remove_list += ['please', 'thank', 'pacific', 'canada',
        'none', 'florida', 'cst', 'tue', 'mon', 'wed', 'thu', 'fri', 'sat',
        'sun', 'cd', 'gmt', 'hw', 'sw', 'sg']
        self.spelling_map = {
            'gantri': 'gantry',
            'gantree': 'gantry',
            'ystem': 'system',
        }

    def process_text(self, text):
        comp = re.compile(r'Image:[\S]*')
        text = comp.sub('', text)
        terms = self.tokenize(self.clean_text(text).lower().strip())
        clean_terms = [term for term in terms if term not in self.remove_list and len(term) > 1]
        stemmed_tokens = list(self.porter_stem(clean_terms))
        # remove weird punctuation and numbers and images
        return ' '.join(stemmed_tokens)

    def clean_text(self, text):
        final_text = ''
        for i in text:
            if i == ' ':
                final_text += i
            elif not i.isalpha():
                final_text += ' '
            else:
                final_text += i
        return final_text

    def tokenize(self, text):
        terms = []
        # tokenize sentences before words
        sentences = nltk.sent_tokenize(text)
        for sent in sentences:
            terms += nltk.word_tokenize(sent)
        return terms

    def porter_stem(self, terms):
        for term in terms:
            yield self.stemmer.stem(term)


In [56]:
preprocess = Preprocess()
clean_srdf = srdf
clean_srdf['symptom'] = srdf['symptom'].apply(preprocess.process_text)
clean_srdf

,sr_id,Created_Date,Resolution_Code,symptom
0,1-200016966951,10/27/2016 12:51,Reset_UPS,fail come
1,1-202188076151,12/12/2016 0:05,Troubleshoot_HEMIT_TANK,experienc sever artifact follow exam
2,1-186899060621,3/21/2016 20:55,No structural problem found,tech request error log review qa issu th
3,360154703,10/18/2012 0:00,No structural problem found,enabl dose manag custom
4,1-174918638991,7/31/2015 21:25,Configure_ConnectPro,question regard modal worklist chang call john...
5,1-205402405831,2/24/2017 18:26,Troubleshoot_HEMIT_TANK,still get artifact tilt gantri axial head
6,1-184683709341,2/10/2016 14:48,No structural problem found,need ip chang made
7,1-182969359501,1/13/2016 23:59,Reset_the_system_time_clock,ha come attent thi machin wa set name hospit m...
8,1-200863773491,11/15/2016 17:28,Adjust_gantry_tilt_speed,gantr come direct connect
9,1-182424492456,1/6/2016 23:16,Troubleshoot_GANTRY_TOUCH_SENSOR,plate front cover broke


In [57]:
X = clean_srdf.symptom
y = clean_srdf.Resolution_Code


# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [58]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    
    # create document-term matrices using the vectorizer
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
    
    # print the number of features that were generated
    print('Features: ', X_train_dtm.shape[1])
    
    
    
    # use Multinomial Naive Bayes to predict the star rating
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    
    # print the accuracy of its predictions
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))
    return(y_pred_class)

# tune TfidfVectorizer for better results
vect = TfidfVectorizer(norm=None)
tokenize_test(vect)

Features:  3235
Accuracy:  0.40736342042755347


array(['Reset_UPS', 'Clean_or_adjust_mylar_scan_window',
       'Troubleshoot_JEDI_INVERTER_AND_HV_TANK', ...,
       'Clean_or_adjust_mylar_scan_window',
       'Cycle_the_Console_power_or_restart_system_software',
       'No structural problem found'], dtype='<U55')

In [ ]:
# run multinomial / random forest 

In [ ]:
# ensemble voting

In [ ]:
# cross validation